In [1]:
from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("Jupyter").getOrCreate()

# spark = SparkSession.builder.config("spark.jars", "/opt/spark/jars/postgresql-42.7.4.jar")\
# .master("local")\
# .appName("PySpark_Postgres_test2")\
# .getOrCreate() 
spark 

25/03/28 12:24:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
spark.stop()

In [25]:
file_bike_2016='/opt/airflow/data/2016-citibike-tripdata/1_January/*.csv'
file_bike_2013_06='/opt/airflow/include/2013-citibike-tripdata/*/*'
file_bike_2024_01='/home/iceberg/data_2/2024-citibike-tripdata/01/202401-citibike-tripdata.csv'
file_bike_2019_01='/home/iceberg/notebooks/data/2019-citibike-tripdata/1_January/201901-citibike-tripdata_1.csv'

In [27]:
df_2016= spark.read.format('csv').options(inferSchema='True',header='True').load(file_bike_2016)

In [4]:
df2023 =spark.read.format('csv').options(inferSchema='True', header='True',delimiter=',').load(file_bike_2013_06)
# df2013 =spark.read.format('csv').options(inferSchema='True', header='True',delimiter=',').load(file_bike_2013_06)

In [58]:
dict(df_2016.dtypes)

{'trip_duration': 'int',
 'start_at': 'timestamp',
 'stop_at': 'string',
 'start_station_id': 'int',
 'start_station_name': 'string',
 'start_station_latitude': 'double',
 'start_station_longitude': 'double',
 'end_station_id': 'int',
 'end_station_name': 'string',
 'end_station_latitude': 'double',
 'end_station_longitude': 'double',
 'bike_id': 'bigint',
 'user_type': 'string',
 'customer_year_birth': 'double',
 'gender': 'int'}

In [55]:
df_2016.show(2)

+-------------+--------+-----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-------------------+------+
|trip_duration|start_at|          stop_at|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|customer_year_birth|gender|
+-------------+--------+-----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-------------------+------+
|          923|    NULL|1/1/2016 00:16:04|             268|Howard St & Centr...|           40.71910537|           -73.99973337|          3002|South End Ave & L...|           40.711512|           -74.015756|  22285|Subscriber|             195

In [34]:
from pyspark.sql.functions import col
df_2016 = df_2016.withColumn('starttime',col('starttime').cast("timestamp"))

In [51]:
df_2016 = df_2016.withColumn('bike_id', col('bike_id').cast("bigint"))

In [74]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType , TimestampType , DoubleType , FloatType, DateType
from pyspark.sql.functions import when,col,lit,hash,datediff,xxhash64,year,quarter,dayofmonth,month,dayofweek,date_format,concat

bronze_schema_ny_bike = StructType([
    StructField("dw_period_tag", StringType(), nullable=True),
    StructField("start_station_id", StringType(), nullable=True),
    StructField("start_station_name", StringType(), nullable=True),
    StructField("start_station_latitude", DoubleType(), nullable=True),
    StructField("start_station_longitude", DoubleType(), nullable=True),
    StructField("end_station_id", StringType(), nullable=True),
    StructField("end_station_name", StringType(), nullable=True),
    StructField("end_station_latitude", DoubleType(), nullable=True),
    StructField("end_station_longitude", DoubleType(), nullable=True),
    StructField("bike_id", IntegerType(), nullable=True),
    StructField("user_type", StringType(), nullable=True),
    StructField("gender", StringType(), nullable=True),
    StructField("customer_year_birth", StringType(), nullable=True),
    StructField("start_at", TimestampType(), nullable=True),
    StructField("stop_at", TimestampType(), nullable=True),
    StructField("trip_duration", DoubleType(), nullable=True)
])

In [47]:
df_casted = df_2016.select([
    col(field.name).cast(field.dataType).alias(field.name) 
    for field in bronze_schema_ny_bike.fields
])

In [122]:
print(bronze_schema_ny_bike)

StructType([StructField('dw_period_tag', StringType(), True), StructField('start_station_id', StringType(), True), StructField('start_station_name', StringType(), True), StructField('start_station_latitude', DoubleType(), True), StructField('start_station_longitude', DoubleType(), True), StructField('end_station_id', StringType(), True), StructField('end_station_name', StringType(), True), StructField('end_station_latitude', DoubleType(), True), StructField('end_station_longitude', DoubleType(), True), StructField('bike_id', IntegerType(), True), StructField('user_type', StringType(), True), StructField('gender', StringType(), True), StructField('customer_year_birth', StringType(), True), StructField('start_at', TimestampType(), True), StructField('stop_at', TimestampType(), True), StructField('trip_duration', DoubleType(), True)])


In [98]:
data_model_list = bronze_schema_ny_bike.names

In [95]:
data_model_list

['dw_period_tag',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'gender',
 'customer_year_birth',
 'start_at',
 'stop_at',
 'trip_duration']

In [107]:
data_model_df = list(dict(df_2016.dtypes).keys())

In [108]:
data_model_df

['trip_duration',
 'start_at',
 'stop_at',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'customer_year_birth',
 'gender']

In [111]:
filtered = list(filter( lambda x: x not in data_model_df ,data_model_list ))

In [112]:
filtered

['dw_period_tag']

In [71]:
df_2016.withColumn('test_column',lit(None)).show(2)

+-------------+--------+-----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-------------------+------+-----------+
|trip_duration|start_at|          stop_at|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|customer_year_birth|gender|test_column|
+-------------+--------+-----------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-------------------+------+-----------+
|          923|    NULL|1/1/2016 00:16:04|             268|Howard St & Centr...|           40.71910537|           -73.99973337|          3002|South End Ave & L...|           40.711512|           -74.015756

In [16]:
df2023.groupBy().agg(
            current_timestamp().alias('date'),
            count(expr("*")).alias('number_of_rows'),
            lit('process_name').alias('process_name'),
            lit('OK').alias('status'),
            # year('starttime').alias('YEAR'),
            # month('starttime').alias('MONTH')
        ).show()

+--------------------+--------------+------------+------+
|                date|number_of_rows|process_name|status|
+--------------------+--------------+------------+------+
|2025-01-11 20:52:...|       5614929|process_name|    OK|
+--------------------+--------------+------------+------+



In [7]:

from datetime import datetime
from pyspark.sql.functions import count,expr,current_timestamp,lit,month,year

startime_row = df2023.take(1)[0]
# Parse the string into a datetime object
date_obj = datetime.strptime(startime_row.asDict()['starttime'], "%Y-%m-%d %H:%M:%S")
# Extract the month
month = date_obj.month
year = date_obj.year

month
year 

# df2023.select((current_timestamp()).alias('date')\
#                         ,count(expr("*")).alias('number_of_rows')\
#                         ,(lit('process_name')).alias('process_name')\
#                         ,(lit('OK')).alias('status')
#                         ,(lit(month)).alias('YEAR')
#                         ,(lit(year)).alias('MONTH')
#                         ).show()

2013

In [2]:
df_fact = spark.read\
            .format("jdbc") \
                .option("url", "jdbc:postgresql://db:5432/postgres") \
                .option("dbtable", "fact_nybike") \
                .option("user", "root") \
                .option("password", "your-password") \
                .option("driver", "org.postgresql.Driver") \
                .option("fetchsize",1000) \
                .load()

In [8]:
df_station = spark.read\
            .format("jdbc") \
                .option("url", "jdbc:postgresql://db:5432/postgres") \
                .option("dbtable", "dim_station_nybike") \
                .option("user", "root") \
                .option("password", "your-password") \
                .option("driver", "org.postgresql.Driver") \
                .option("fetchsize",1000) \
                .load()

In [15]:
df_report = spark.read\
            .format("jdbc") \
                .option("url", "jdbc:postgresql://db:5432/postgres") \
                .option("dbtable", "report_process") \
                .option("user", "root") \
                .option("password", "your-password") \
                .option("driver", "org.postgresql.Driver") \
                .option("fetchsize",1000) \
                .load()

In [16]:
db_table=df_fact.createOrReplaceTempView('dim_fact')
df_station.createOrReplaceTempView('dim_station')
df_report.createOrReplaceTempView('dim_report')

In [21]:
list_row=spark.sql("SELECT * FROM dim_report WHERE status=='OK'").collect()

In [24]:
list_row[0]['year_collected']

2013

In [34]:
# spark.sql(f"SELECT * FROM dim_station WHERE YEAR={list_row[0]['year_collected']} AND MONTH={list_row[0]['month_collected']}").show()

spark.sql(f"""SELECT * FROM dim_station WHERE YEAR={list_row[0]['year_collected']} {"AND MONTH={list_row[0]['month_collected']}" if list_row[0]['month_collected']>0 else" "}""").show()


+----------------------+----+-----+--------------------+----------------+----------------------+-----------------------+--------------------+--------------+--------------------+---------------------+
|TRIP_STATION_NYBIKE_ID|YEAR|MONTH|  START_STATION_NAME|START_STATION_ID|START_STATION_LATITUDE|START_STATION_LONGITUDE|    END_STATION_NAME|END_STATION_ID|END_STATION_LATITUDE|END_STATION_LONGITUDE|
+----------------------+----+-----+--------------------+----------------+----------------------+-----------------------+--------------------+--------------+--------------------+---------------------+
|  -8210704430849674546|2013|   12|            State St|             427|           40.70251526|           -74.01427023|Rivington St & Ch...|           473|         40.72110063|          -73.9919254|
|  -5868319784827588472|2013|   12|Centre St & Worth St|             386|           40.71494807|           -74.00234482|Columbia Heights ...|           216|         40.70037867|         -73.99548059|


In [1]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import uuid
import os
# Function to list all CSV files recursively
def list_csv_files(directory):
    csv_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    return csv_files

In [11]:
files = os.walk('/opt/airflow/data/2013-citibike-tripdata')

In [12]:
for root ,dir_names,filename in files:
    print(f"root : {root} , dir_names: {dir_names} , filename : {filename}" )

root : /opt/airflow/data/2013-citibike-tripdata , dir_names: ['12_December', '7_July', '8_August', '9_September', '10_October', '11_November', '5_May', '6_June'] , filename : ['.DS_Store']
root : /opt/airflow/data/2013-citibike-tripdata/12_December , dir_names: [] , filename : ['201312-citibike-tripdata_1.csv']
root : /opt/airflow/data/2013-citibike-tripdata/7_July , dir_names: [] , filename : ['201307-citibike-tripdata_1.csv']
root : /opt/airflow/data/2013-citibike-tripdata/8_August , dir_names: [] , filename : ['201308-citibike-tripdata_2.csv', '201308-citibike-tripdata_1.csv']
root : /opt/airflow/data/2013-citibike-tripdata/9_September , dir_names: [] , filename : ['201309-citibike-tripdata_2.csv', '201309-citibike-tripdata_1.csv']
root : /opt/airflow/data/2013-citibike-tripdata/10_October , dir_names: [] , filename : ['201310-citibike-tripdata_1.csv', '201310-citibike-tripdata_2.csv']
root : /opt/airflow/data/2013-citibike-tripdata/11_November , dir_names: [] , filename : ['201311-

In [2]:
path='/opt/airflow/data/2016-citibike-tripdata'
list_files =list_csv_files(path)

print(list_files)

['/opt/airflow/data/2016-citibike-tripdata/1_January/201601-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/12_December/201612-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/2_February/201602-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/7_July/201607-citibike-tripdata_2.csv', '/opt/airflow/data/2016-citibike-tripdata/7_July/201607-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/3_March/201603-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/8_August/201608-citibike-tripdata_2.csv', '/opt/airflow/data/2016-citibike-tripdata/8_August/201608-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/4_April/201604-citibike-tripdata_2.csv', '/opt/airflow/data/2016-citibike-tripdata/4_April/201604-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/9_September/201609-citibike-tripdata_2.csv', '/opt/airflow/data/2016-citibike-tripdata/9_September/201609-citibike-tri

In [3]:

group_size = 5
len(list_files)
for i in range(0, len(list_files), group_size):
    group_files = list_files[i:i + group_size]
    print(f"group {i}",group_files)
    # df = spark.read.csv(group_files , header=True, inferSchema=False)
    # df.show(8)

group 0 ['/opt/airflow/data/2016-citibike-tripdata/1_January/201601-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/12_December/201612-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/2_February/201602-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/7_July/201607-citibike-tripdata_2.csv', '/opt/airflow/data/2016-citibike-tripdata/7_July/201607-citibike-tripdata_1.csv']
group 5 ['/opt/airflow/data/2016-citibike-tripdata/3_March/201603-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/8_August/201608-citibike-tripdata_2.csv', '/opt/airflow/data/2016-citibike-tripdata/8_August/201608-citibike-tripdata_1.csv', '/opt/airflow/data/2016-citibike-tripdata/4_April/201604-citibike-tripdata_2.csv', '/opt/airflow/data/2016-citibike-tripdata/4_April/201604-citibike-tripdata_1.csv']
group 10 ['/opt/airflow/data/2016-citibike-tripdata/9_September/201609-citibike-tripdata_2.csv', '/opt/airflow/data/2016-citibike-tripdata/9_Se

In [3]:
df.show()

+-----------+-------------------------+-----------+----+-----+------------+-------------+---------------+-------------------+-------------+
|    FACT_ID|TRIP_STATION_NYBIKE_ID_FK| DATE_ID_FK|YEAR|MONTH|RIDABLE_TYPE|CUSTOMER_TYPE|CUSTOMER_GENDER|CUSTOMER_YEAR_BIRTH|TRIP_DURATION|
+-----------+-------------------------+-----------+----+-----+------------+-------------+---------------+-------------------+-------------+
| 1649138811|               1649138811| 1649138811|2013|    6| classe_bike|       Member|         Female|             1960.0|         1770|
|-1778667760|              -1778667760|-1778667760|2013|    6| classe_bike|       Member|           Male|             1968.0|         3541|
|-1523717471|              -1523717471|-1523717471|2013|    6| classe_bike|       Member|           Male|             1986.0|          861|
|  106814874|                106814874|  106814874|2013|    6| classe_bike|       Member|         Female|             1970.0|         1396|
| 2074657112|       

In [9]:
from pyspark.sql.functions import col,hash
df.groupBy("FACT_ID").count().filter(col("count") > 1).show() 
# columns=['TRIP_STATION_NYBIKE_ID_FK','DATE_ID_FK','RIDABLE_TYPE','TRIP_DURATION']
# df.withColumn("ID_COL",hash(*columns )).limit(20).show()

25/01/26 21:05:37 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-----------+-----+
|    FACT_ID|count|
+-----------+-----+
| -450014632|    2|
|-2032378884|    2|
|-1978845173|    2|
|-1955060958|    2|
|-1888077861|    2|
|-1887046999|    2|
|-1761519587|    2|
|-1626918705|    2|
|-1472320072|    2|
|-1407056905|    2|
|-1194473220|    2|
| -965298935|    2|
| -801461793|    2|
| -679464594|    2|
| -638953031|    2|
| -521349326|    2|
| -426100946|    2|
|   89843662|    2|
|  143768786|    2|
|  347295293|    2|
+-----------+-----+
only showing top 20 rows



In [4]:
from pyspark.sql import Row
studentDf = spark.createDataFrame([
    Row(id=1,name='vijay',marks=67),
    Row(id=2,name='Ajay',marks=88),
    Row(id=3,name='jay',marks=79),
    Row(id=4,name='vinay',marks=67),
])
studentDf.show()

+---+-----+-----+
| id| name|marks|
+---+-----+-----+
|  1|vijay|   67|
|  2| Ajay|   88|
|  3|  jay|   79|
|  4|vinay|   67|
+---+-----+-----+



In [6]:
studentDf.select("id","name","marks").write.format("jdbc")\
.option("url", "jdbc:postgresql://db_postgres:5432/postgres")\
.option("driver", "org.postgresql.Driver").option("dbtable", "students")\
.option("user", "root").option("password", "your-password").save()

In [21]:
from pyspark.sql.functions import count,expr,current_timestamp,lit
from datetime import datetime
df2023.select(count(expr("*")).alias('number_of_row'),(current_timestamp()).alias('date'),(lit('ETL-NY-Process-Before-publish')).alias('Name_of_Process')).show()

+-------------+--------------------+--------------------+
|number_of_row|                date|     Name_of_Process|
+-------------+--------------------+--------------------+
|       577703|2025-01-08 12:52:...|ETL-NY-Process-Be...|
+-------------+--------------------+--------------------+



In [26]:
df_result =df2023.select(count(expr("*")).alias('number_of_row'))

In [27]:
df_result.join(df2023.select((current_timestamp()).alias('date'))).show()

+-------------+--------------------+
|number_of_row|                date|
+-------------+--------------------+
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
|       577703|2025-01-08 15:34:...|
+-------------+--------------------+
only showing top 20 rows



In [12]:
from datetime import datetime
datetime.now()

<function datetime.date>

In [49]:
!pip install datetime-quarter


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [150]:
import calendar 
from datequarter import DateQuarter
from pyspark.sql.functions import year,quarter,dayofweek,dayofmonth,month,lit,concat,date_format,hash
# df2023.select(df2023.starttime,df2023.stoptime).show() 
# df2023.withColumns({'YEAR':year('starttime'),'QUARTER':quarter(df2023.starttime),'QUARTER_NAME':concat(year('starttime'), lit('Q'), quarter('starttime'))}).show()
# df2023.select(year('starttime'),quarter('starttime'),concat(year('starttime'), lit('Q'), quarter('starttime')),month('starttime'),date_format('starttime', 'MMMM').alias('MONTH_NAME'),dayofmonth('starttime'),dayofweek('starttime'),date_format('starttime','dd'),(date_format('starttime','EEEE').alias('WEEK_DAY_NAME'))).show()
# df2023.withColumn('ID',hash('starttime','stoptime')).select('*').show()
df2023.drop('startti')

DataFrame[tripduration: int, starttime: timestamp, stoptime: timestamp, start station id: double, start station name: string, start station latitude: double, start station longitude: double, end station id: double, end station name: string, end station latitude: double, end station longitude: double, bikeid: int, usertype: string, birth year: int, gender: int]

In [71]:
!pip install datetime-quarter


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [72]:
from datequarter import DateQuarter
sample = DateQuarter(2019, 4)

<bound method DateQuarter.__str__ of <DateQuarter-2019,Q4>>

In [1]:
%%sql 
-- CREATE DATABASE IF NOT EXISTS STAGE_NEWYORKCITYBIKE
-- CREATE DATABASE IF NOT EXISTS DATAWAHOUSE_NEWCITYBIKE

25/03/04 19:51:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [3]:
%%sql

-- CREATE DATABASE dw_nybike;

USE dw_nybike;
-- CREATE SCHEMA IF NOT EXISTS  bronze;
-- CREATE SCHEMA IF NOT EXISTS sylver ;
-- CREATE SCHEMA IF NOT EXISTS gold ;

AnalysisException: [SCHEMA_NOT_FOUND] The schema `dw_nybike` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a catalog, verify the current_schema() output, or qualify the name with the correct catalog.
To tolerate the error on drop use DROP SCHEMA IF EXISTS.

In [221]:
import uuid

myuuid = uuid.uuid4()

print('Your UUID is: ' + str(myuuid))

Your UUID is: d1d804f8-fa54-4bde-ac07-9d83a669a90b


UUID('d1d804f8-fa54-4bde-ac07-9d83a669a90b')

In [12]:
%%sql
CREATE TABLE dw_nybike.bronze.trip_data_nybike(
    dw_period_tag string,
    ride_id string,
	start_station_id string,
	start_station_name string,
	start_station_latitude DOUBLE ,
	start_station_longitude DOUBLE ,
	end_station_id string,
	end_station_name string,
	end_station_latitude double,
	end_station_longitude double,
	user_type string,
    gender string,
	customer_year_birth string,
    bike_id bigint,
	rideable_type string,
	start_at timestamp,
	stop_at timestamp, 
	trip_duration DOUBLE)
PARTITIONED BY (dw_period_tag)
;


++
||
++
++

In [2]:
%%sql
select 
    -- distinct(dw_period_tag)
    count(*)
    from dw_nybike.bronze.trip_data_nybike
    -- WHERE dw_period_tag=
    limit 5; 

-- truncate table dw_nybike.sylver.trip_data_nybike;


AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `dw_nybike`.`bronze`.`trip_data_nybike` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 4 pos 9;
'GlobalLimit 5
+- 'LocalLimit 5
   +- 'Aggregate [unresolvedalias(count(1), None)]
      +- 'UnresolvedRelation [dw_nybike, bronze, trip_data_nybike], [], false


In [167]:
%%sql
-- SHOW tables dw_nybike;
-- use dw_nybike.bronze;
use dw_nybike.sylver;
-- show tables  ;

++
||
++
++

In [135]:
df_2017 = spark.sql(""" select * from demo.dw_nybike.bronze.trip_data_nybike where dw_period_tag='2014' """) 

In [136]:
df_2017.show(3)

+-------------+-------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+--------+-------+-------------+
|dw_period_tag|ride_id|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude| user_type|gender|customer_year_birth|bike_id|rideable_type|start_at|stop_at|trip_duration|
+-------------+-------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+----------+------+-------------------+-------+-------------+--------+-------+-------------+
|         2014|   NULL|             479|     9 Ave & W 45 St|           40.76019252|            -73.9912551|           540|Lexington Ave & E...|         40.7414728

In [19]:
types_df = df.dtypes

In [22]:
dict_types_df= dict(types_df)

In [23]:
dict_types_df

{'dw_period_tag': 'string',
 'ride_id': 'string',
 'start_station_id': 'string',
 'start_station_name': 'string',
 'start_station_latitude': 'double',
 'start_station_longitude': 'double',
 'end_station_id': 'string',
 'end_station_name': 'string',
 'end_station_latitude': 'double',
 'end_station_longitude': 'double',
 'user_type': 'string',
 'gender': 'string',
 'customer_year_birth': 'string',
 'bike_id': 'bigint',
 'rideable_type': 'string',
 'start_at': 'timestamp',
 'stop_at': 'timestamp',
 'trip_duration': 'double'}

In [205]:
%%sql
    
CREATE TABLE dw_nybike.sylver.trip_data_nybike(
    trip_uuid string,
    dw_period_tag string,
	start_station_id string,
	start_station_name string,
	start_station_latitude double,
	start_station_longitude double,
	end_station_id string,
	end_station_name string,
	end_station_latitude double,
	end_station_longitude double,
	bike_id string,
    customer_type string,
    enr_gender string,
	customer_year_birth  string,
	rideable_type string,
	start_at timestamp,
	stop_at timestamp,
	trip_duration double,
    quarter int,
    quarter_name string,
    month int,
    month_name string,
    day int,
    weekday int,
    weekday_name string
    )
    PARTITIONED BY (dw_period_tag)
    ;

++
||
++
++

In [206]:
%%sql 
select 
     count(*)
from  dw_nybike.sylver.trip_data_nybike
limit 5;

trip_uuid,dw_period_tag,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,customer_type,enr_gender,customer_year_birth,rideable_type,start_at,stop_at,trip_duration,quarter,quarter_name,month,month_name,day,weekday,weekday_name
None,2013,254,W 11 St & 6 Ave,40.73532427,-73.99800419,147,Greenwich St & Warren St,40.71542197,-74.01121978,15014,Member,Male,1974,classic_bike,2013-09-01 00:00:02,2013-09-01 00:16:52,1010.0,3,2013Q3,9,September,1,1,Sunday
None,2013,151,Cleveland Pl & Spring St,40.7218158,-73.99720307,497,E 17 St & Broadway,40.73704984,-73.99009296,19393,Customer,Unknown,\N,classic_bike,2013-09-01 00:00:09,2013-09-01 00:24:12,1443.0,3,2013Q3,9,September,1,1,Sunday
None,2013,352,W 56 St & 6 Ave,40.76340613,-73.97722479,405,Washington St & Gansevoort St,40.739323,-74.008119,16160,Member,Male,1992,classic_bike,2013-09-01 00:00:16,2013-09-01 00:23:23,1387.0,3,2013Q3,9,September,1,1,Sunday
None,2013,490,8 Ave & W 33 St,40.751551,-73.993934,459,W 20 St & 11 Ave,40.746745,-74.007756,14997,Member,Male,1973,classic_bike,2013-09-01 00:00:18,2013-09-01 00:07:03,405.0,3,2013Q3,9,September,1,1,Sunday
None,2013,236,St Marks Pl & 2 Ave,40.7284186,-73.98713956,393,E 5 St & Avenue C,40.72299208,-73.97995466,19609,Member,Male,1984,classic_bike,2013-09-01 00:00:20,2013-09-01 00:04:50,270.0,3,2013Q3,9,September,1,1,Sunday


In [23]:
df2013.write.mode('overwrite').saveAsTable('NEWYORKCITYBIKE.bike2013')
df2023.write.mode('overwrite').saveAsTable('NEWYORKCITYBIKE.bike2023')

In [14]:
%%sql
-- use PRODUCTION_NEWYORKCITYBIKE;
    select  count(*)   from PRODUCTION_NEWYORKCITYBIKE.FACT_NYBIKE limit 50


count(1)
7093596


In [2]:
%%sql
show tables in STAGE_NEWYORKCITYBIKE

namespace,tableName,isTemporary
STAGE_NEWYORKCITYBIKE,DIM_DATE_NYBIKE,False
STAGE_NEWYORKCITYBIKE,DIM_STATION_NYBIKE,False
STAGE_NEWYORKCITYBIKE,FACT_NYBIKE,False
STAGE_NEWYORKCITYBIKE,student,False


In [ ]:
%%sql 

CREATE TABLE IF NOT EXISTS PRODUCTION_NEWYORKCITYBIKE.DIM_DATE_NYBIKE (
    DATE_ID int,
    YEAR int,
    QUARTER int,
    QUATERNAME string,
    MONTH int,
    MONTHNAME string,
    DAY int,
    WEEKDAY int,
    WEEKDAYNAME string,
    START_AT timestamp,
    STOP_AT timestamp
    ) 
    PARTITIONED BY (month(START_AT)) 
    USING iceberg;

CREATE TABLE IF NOT EXISTS PRODUCTION_NEWYORKCITYBIKE.DIM_STATION_NYBIKE(
    TRIP_ID string,
    START_STATION_NAME string,
    START_STATION_ID int,
    START_STATION_LATITUDE double,
    START_STATION_LONGITUDE double,
    END_STATION_ID int,
    END_STATION_LATITUDE  double,
    END_STATION_LONGITUDE double,
    END_STATION_NAME string,
    END_STATION_ID int,
    -- START_NEIGHBORHOOD string,
    -- END_NEIGHBORHOOD string,
    TIME timestamp
    ) 
    PARTITIONED BY ((month(TIME)) 
    USING iceberg;
    
CREATE TABLE IF NOT EXISTS PRODUCTION_NEWYORKCITYBIKE.FACT_NYBIKE(
    TRIP_ID string,
    RIDEABLE_TYPE string,
    CUSTOMER_TYPE string,
    CUSTOMER_GENDER string,
    CUSTOMER_YEAR_BIRTH int,
    TRIP_DURATION int,
    TIME timestamp
    )
    USING iceberg
    PARTITIONED BY (month(TIME));

In [208]:
import pyspark.sql.functions as sf
spark.range(1).select(
    sf.java_method(
        sf.lit("java.util.UUID"),
        sf.lit("fromString"),
        sf.lit("a5cf6c42-0c85-418f-af6c-3e4e5b1328f2")
    )
).show(truncate=False)

+-----------------------------------------------------------------------------+
|java_method(java.util.UUID, fromString, a5cf6c42-0c85-418f-af6c-3e4e5b1328f2)|
+-----------------------------------------------------------------------------+
|a5cf6c42-0c85-418f-af6c-3e4e5b1328f2                                         |
+-----------------------------------------------------------------------------+



In [214]:
from pyspark.sql.functions import uuid

df_2013.withColumn("uuid",uuid()).show(5)

ImportError: cannot import name 'uuid' from 'pyspark.sql.functions' (/opt/spark/python/pyspark/sql/functions.py)

In [56]:
df2013.show(n=2,vertical=True)

-RECORD 0--------------------------------------
 tripduration            | 695                 
 starttime               | 2013-06-01 00:00:01 
 stoptime                | 2013-06-01 00:11:36 
 start station id        | 444                 
 start station name      | Broadway & W 24 St  
 start station latitude  | 40.7423543          
 start station longitude | -73.98915076        
 end station id          | 434.0               
 end station name        | 9 Ave & W 18 St     
 end station latitude    | 40.74317449         
 end station longitude   | -74.00366443        
 bikeid                  | 19678               
 usertype                | Subscriber          
 birth year              | 1983.0              
 gender                  | 1                   
 RIDEBLE_TYPE            | classic_bike        
-RECORD 1--------------------------------------
 tripduration            | 693                 
 starttime               | 2013-06-01 00:00:08 
 stoptime                | 2013-06-01 00

df2023.write.mode('overwrite').saveAsTable('NEWYORKCITYBIKE.bike2023')
df2013.write.mode('overwrite').saveAsTable('NEWYORKCITYBIKE.bike2013')

In [55]:
from pyspark.sql.functions import col,lit
df2013 = df2013.withColumn('RIDEBLE_TYPE',lit('classic_bike'))

In [58]:
df2013.withColumnsRenamed({'RIDEBLE_TYPE':'RIDEBLE_TYPE'})

DataFrame[tripduration: int, starttime: timestamp, stoptime: timestamp, start station id: int, start station name: string, start station latitude: double, start station longitude: double, end station id: double, end station name: string, end station latitude: double, end station longitude: double, bikeid: int, usertype: string, birth year: double, gender: int, RIDEBLE_TYPE: string]

In [19]:
%%sql 

select * from STAGE_NEWYORKCITYBIKE.BigTable
limit 5

TRIP_DURATION,START_AT,STOP_AT,START_STATION_ID,START_STATION_NAME,START_STATION_LATITUDE,START_STATION_LONGITUDE,END_STATION_ID,END_STATION_NAME,END_STATION_LATITUDE,END_STATION_LONGITUDE,BIKE_ID,USER_TYPE,BIRTH_YEAR,GENDER,RIDABLE_TYPE
326,2013-10-01 00:01:08,2013-10-01 00:06:34,239,Willoughby St & Fleet St,40.69196566,-73.9813018,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,16052,Subscriber,1982,Male,classe_bike
729,2013-10-01 00:01:21,2013-10-01 00:13:30,322,Clinton St & Tillary St,40.696192,-73.991218,398.0,Atlantic Ave & Furman St,40.69165183,-73.9999786,19412,Customer,\N,Unknown,classe_bike
520,2013-10-01 00:01:24,2013-10-01 00:10:04,174,E 25 St & 1 Ave,40.7381765,-73.97738662,403.0,E 2 St & 2 Ave,40.72502876,-73.99069656,19645,Subscriber,1984,Male,classe_bike
281,2013-10-01 00:01:25,2013-10-01 00:06:06,430,York St & Jay St,40.7014851,-73.98656928,323.0,Lawrence St & Willoughby St,40.69236178,-73.98631746,16992,Subscriber,1985,Male,classe_bike
196,2013-10-01 00:01:27,2013-10-01 00:04:43,403,E 2 St & 2 Ave,40.72502876,-73.99069656,401.0,Allen St & Rivington St,40.72019576,-73.98997825,15690,Subscriber,1986,Male,classe_bike


In [15]:
%%sql 
drop table STAGE_NEWYORKCITYBIKE.BigTable

++
||
++
++

In [35]:
df2023.printSchema()

root
 |-- ride_id: string (nullable = true)
 |-- rideable_type: string (nullable = true)
 |-- started_at: timestamp (nullable = true)
 |-- ended_at: timestamp (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- start_lat: double (nullable = true)
 |-- start_lng: double (nullable = true)
 |-- end_lat: double (nullable = true)
 |-- end_lng: double (nullable = true)
 |-- member_casual: string (nullable = true)



In [24]:
df2013.printSchema()

root
 |-- tripduration: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- stoptime: timestamp (nullable = true)
 |-- start station id: integer (nullable = true)
 |-- start station name: string (nullable = true)
 |-- start station latitude: double (nullable = true)
 |-- start station longitude: double (nullable = true)
 |-- end station id: string (nullable = true)
 |-- end station name: string (nullable = true)
 |-- end station latitude: string (nullable = true)
 |-- end station longitude: string (nullable = true)
 |-- bikeid: integer (nullable = true)
 |-- usertype: string (nullable = true)
 |-- birth year: string (nullable = true)
 |-- gender: integer (nullable = true)



In [25]:
df2023.groupBy("started_at","ended_at").count().filter("count > 1").show()

+----------+--------+-----+
|started_at|ended_at|count|
+----------+--------+-----+
+----------+--------+-----+



In [43]:
# list of transformations
from pyspark.sql.functions import col, when
df2013.withColumn("genre",when(col('gender') == 1,"Male").when(col('gender') == 2,'Female').otherwise('Unknown')).show(n=2,vertical=True)

-RECORD 0--------------------------------------
 tripduration            | 695                 
 starttime               | 2013-06-01 00:00:01 
 stoptime                | 2013-06-01 00:11:36 
 start station id        | 444                 
 start station name      | Broadway & W 24 St  
 start station latitude  | 40.7423543          
 start station longitude | -73.98915076        
 end station id          | 434                 
 end station name        | 9 Ave & W 18 St     
 end station latitude    | 40.74317449         
 end station longitude   | -74.00366443        
 bikeid                  | 19678               
 usertype                | Subscriber          
 birth year              | 1983                
 gender                  | 1                   
 genre                   | Male                
-RECORD 1--------------------------------------
 tripduration            | 693                 
 starttime               | 2013-06-01 00:00:08 
 stoptime                | 2013-06-01 00

In [52]:
from pyspark.sql.functions import datediff
df2023 = df2023.withColumn('trip_duration', datediff (df2023.ended_at , df2023.started_at))

In [59]:
df2023.show(n=2, vertical=True)

-RECORD 0----------------------------------
 ride_id            | DC1CB984821DFFF7     
 rideable_type      | classic_bike         
 started_at         | 2023-01-07 15:36:... 
 ended_at           | 2023-01-07 15:39:... 
 start_station_name | Vesey St & Church St 
 start_station_id   | 5216.06              
 end_station_name   | Albany St & Green... 
 end_station_id     | 5145.02              
 start_lat          | 40.71221958009051    
 start_lng          | -74.01047229766846   
 end_lat            | 40.709267            
 end_lng            | -74.013247           
 member_casual      | member               
-RECORD 1----------------------------------
 ride_id            | C00CA02971557F16     
 rideable_type      | classic_bike         
 started_at         | 2023-01-04 19:23:... 
 ended_at           | 2023-01-04 19:34:... 
 start_station_name | Lispenard St & Br... 
 start_station_id   | 5391.06              
 end_station_name   | St Marks Pl & 1 Ave  
 end_station_id     | 5626.13   

In [62]:
if 'TRIP_DURATION' in df2023.columns: print('yes')

In [62]:
from pyspark.sql import DataFrame
def add_duration_column(df:DataFrame,start_date:str,ended_date:str):
    df=df.withColumn('trip_duration', datediff (ended_date , start_date))
    return df

In [2]:
!pip install geopy 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 5.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from geopy.distance import geodesic

# Define the two points
coord1 = (40.71939226,-74.00247214)  # Berlin
coord2 = (40.72779126,-73.98564945)  # New York

# Calculate the distance
distance = geodesic(coord1, coord2).km

print(f"The distance between Berlin and New York is approximately {distance:.2f} kilometers.")

The distance between Berlin and New York is approximately 1.70 kilometers.


In [4]:
distance

1.6999853938271692

In [32]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="benesh-app")
location = geolocator.reverse(coord1)
# print(location.address)
print(location.raw)


{'place_id': 331190618, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 3708656643, 'lat': '40.7193923', 'lon': '-74.0024721', 'class': 'amenity', 'type': 'bicycle_rental', 'place_rank': 30, 'importance': 9.175936522464359e-05, 'addresstype': 'amenity', 'name': 'Citi Bike - Lispenard St & Broadway', 'display_name': 'Citi Bike - Lispenard St & Broadway, Lispenard Street, Tribeca, Manhattan, New York County, City of New York, New York, 10013, United States', 'address': {'amenity': 'Citi Bike - Lispenard St & Broadway', 'road': 'Lispenard Street', 'neighbourhood': 'Tribeca', 'suburb': 'Manhattan', 'county': 'New York County', 'city': 'City of New York', 'state': 'New York', 'ISO3166-2-lvl4': 'US-NY', 'postcode': '10013', 'country': 'United States', 'country_code': 'us'}, 'boundingbox': ['40.7193423', '40.7194423', '-74.0025221', '-74.0024221']}


In [33]:
location

Location(Citi Bike - Lispenard St & Broadway, Lispenard Street, Tribeca, Manhattan, New York County, City of New York, New York, 10013, United States, (40.7193923, -74.0024721, 0.0))

In [7]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()

conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.3')
conf.set('spark.hadoop.fs.s3a.access.key', ACCESS_KEY_1)
conf.set('spark.hadoop.fs.s3a.secret.key',SECRET_KEY_1)
conf.set('spark.hadoop.fs.s3a.path.style.access','true')
conf.set('spark.hadoop.fs.s3a.ssl.enabled','false')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider','org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem')
conf.set("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") 
conf.set('spark.hadoop.fs.s3a.committer.magic.enabled','true')
conf.set('spark.hadoop.fs.s3a.committer.name','magic')


spark = SparkSession.builder.config(conf=conf).getOrCreate()

spark

In [8]:
BUCKET_NAME = 'my-bucket-testomar'
CSV_INPUT_FILE_PATH = f's3a://{BUCKET_NAME}/202402-citibike-tripdata_1.csv'

In [9]:
df2023 =spark.read.format('csv').options(inferSchema='True', header='True',delimiter=',').load(CSV_INPUT_FILE_PATH)

Py4JJavaError: An error occurred while calling o151.load.
: java.lang.NoSuchMethodError: 'java.util.EnumSet org.apache.hadoop.conf.Configuration.getEnumSet(java.lang.String, java.lang.Class, boolean)'
	at org.apache.hadoop.fs.impl.FlagSet.buildFlagSet(FlagSet.java:323)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:743)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:53)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:366)
	at org.apache.spark.sql.DataFrameReader.loadV1Sour

In [10]:
import yaml
import os
from pyspark.sql import SparkSession



In [17]:

# Load YAML config
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

# Fetch secrets from environment variables
source_config = config["source"]
# source_config["password"] = os.environ["SOURCE_DB_PASSWORD"]  # Inject password

etl_conf= config["etl_conf"]

dest_config = config["destination"]
# dest_config["password"] = os.environ["DEST_DB_PASSWORD"]

In [14]:
source_config

{'url': 'jdbc:postgresql://source-host:5432/db',
 'user': 'user',
 'driver': 'org.postgresql.Driver',
 'table': 'source_table'}

In [18]:
etl_conf

{'columnsto_rename': {'tripduration': 'TRIP_DURATION',
  'starttime': 'START_AT',
  'stoptime': 'STOP_AT',
  'start station id': 'START_STATION_ID',
  'start station name': 'START_STATION_NAME',
  'start station latitude': 'START_STATION_LATITUDE',
  'start station longitude': 'START_STATION_LONGITUDE',
  'end station id': 'END_STATION_ID',
  'end station name': 'END_STATION_NAME',
  'end station latitude': 'END_STATION_LATITUDE',
  'end station longitude': 'END_STATION_LONGITUDE',
  'bikeid': 'BIKE_ID',
  'usertype': 'USER_TYPE',
  'birth year': 'CUSTOMER_YEAR_BIRTH',
  'rideable_type': 'RIDEABLE_TYPE',
  'started_at': 'START_AT',
  'ended_at': 'STOP_AT',
  'start_station_name': 'START_STATION_NAME',
  'start_station_id': 'START_STATION_ID',
  'end_station_name': 'END_STATION_NAME',
  'end_station_id': 'END_STATION_ID',
  'start_lat': 'START_STATION_LATITUDE',
  'start_lng': 'START_STATION_LONGITUDE',
  'end_lat': 'END_STATION_LATITUDE',
  'end_lng': 'END_STATION_LONGITUDE',
  'member

In [23]:
from typing import List, Optional

from sqlmodel import Field, Session, SQLModel, create_engine, select

# PostgreSQL connection details (REPLACE THESE WITH YOUR CREDENTIALS)
PG_USER = "root"
PG_PASSWORD = "your-password"
PG_HOST = "db"  # e.g., localhost, 127.0.0.1
PG_PORT = "5432" # default port for postgresql
PG_DATABASE = "postgres"

PG_URL = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    secret_name: str
    age: Optional[int] = Field(default=None, index=True)
    team_id: Optional[int] = Field(default=None, foreign_key="team.id")


class Team(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str = Field(index=True)
    headquarters: str


engine = create_engine(PG_URL)  # Use PostgreSQL URL

def create_db_and_tables():
    SQLModel.metadata.create_all(engine)


def create_heroes_and_teams():
    team_preventers = Team(name="Preventers", headquarters="Sharp Tower")
    team_z_force = Team(name="Z-Force", headquarters="Sister Margaret’s Bar")

    hero_deadpond = Hero(name="Deadpond", secret_name="Dive Wilson", age=22, team_id=team_z_force.id)
    hero_rusty_man = Hero(name="Rusty-Man", secret_name="Tommy Sharp", age=48, team_id=team_preventers.id)
    hero_spider_boy = Hero(name="Spider-Boy", secret_name="Pedro Parqueador", age=17)
    hero_dr_weird = Hero(name="Dr. Weird", secret_name="Steve Weird", age=36, team_id=team_preventers.id)
    hero_captain_north_america = Hero(name="Captain North America", secret_name="Esteban Rogelios", age=93)

    with Session(engine) as session:
        session.add(team_preventers)
        session.add(team_z_force)
        session.add(hero_deadpond)
        session.add(hero_rusty_man)
        session.add(hero_spider_boy)
        session.add(hero_dr_weird)
        session.add(hero_captain_north_america)
        session.commit()
        session.refresh(team_preventers)
        session.refresh(team_z_force)
        session.refresh(hero_deadpond)
        session.refresh(hero_rusty_man)
        session.refresh(hero_spider_boy)
        session.refresh(hero_dr_weird)
        session.refresh(hero_captain_north_america)


def generate_report():
    with Session(engine) as session:
        statement = select(Hero, Team).join(Team, isouter=True)
        results = session.exec(statement)

        print("Heroes Report:")
        for hero, team in results:
            team_name = team.name if team else "No Team"
            print(f"- {hero.name} ({hero.secret_name}), Age: {hero.age or 'Unknown'}, Team: {team_name}")


def query_heroes():
    with Session(engine) as session:
        heroes = session.exec(select(Hero)).all()
        print("\nAll Heroes:")
        for hero in heroes:
            print(hero)

        old_heroes = session.exec(select(Hero).where(Hero.age > 30)).all()
        print("\nHeroes Older Than 30:")
        for hero in old_heroes:
            print(hero)

        preventers_heroes = session.exec(select(Hero).join(Team).where(Team.name == "Preventers")).all()
        print("\nPreventers Heroes:")
        for hero in preventers_heroes:
            print(hero)

        heroes_without_team = session.exec(select(Hero).where(Hero.team_id == None)).all()
        print("\nHeroes without team:")
        for hero in heroes_without_team:
            print(hero)

def main():
    create_db_and_tables()
    create_heroes_and_teams()
    generate_report()
    query_heroes()


if __name__ == "__main__":
    main()

Heroes Report:
- Deadpond (Dive Wilson), Age: 22, Team: No Team
- Rusty-Man (Tommy Sharp), Age: 48, Team: No Team
- Spider-Boy (Pedro Parqueador), Age: 17, Team: No Team
- Dr. Weird (Steve Weird), Age: 36, Team: No Team
- Captain North America (Esteban Rogelios), Age: 93, Team: No Team

All Heroes:
name='Deadpond' team_id=None id=1 age=22 secret_name='Dive Wilson'
name='Rusty-Man' team_id=None id=2 age=48 secret_name='Tommy Sharp'
name='Spider-Boy' team_id=None id=3 age=17 secret_name='Pedro Parqueador'
name='Dr. Weird' team_id=None id=4 age=36 secret_name='Steve Weird'
name='Captain North America' team_id=None id=5 age=93 secret_name='Esteban Rogelios'

Heroes Older Than 30:
name='Rusty-Man' team_id=None id=2 age=48 secret_name='Tommy Sharp'
name='Dr. Weird' team_id=None id=4 age=36 secret_name='Steve Weird'
name='Captain North America' team_id=None id=5 age=93 secret_name='Esteban Rogelios'

Preventers Heroes:

Heroes without team:
name='Deadpond' team_id=None id=1 age=22 secret_name

In [35]:
from sqlmodel import SQLModel, Field
from datetime import datetime
from sqlalchemy.sql import func


class Event(SQLModel, table=True):
    id: int = Field(default=None, primary_key=True)
    name: str
    description: str
    event_time: datetime

    # Automatically set the creation timestamp
    created_at: datetime = Field(default_factory=datetime.utcnow, nullable=False)

    # Automatically update the timestamp on each update
    updated_at: datetime = Field(
        sa_column_kwargs={"server_default": func.now(), "onupdate": func.now()},
        default_factory=datetime.utcnow,
        nullable=False
    )    

PG_USER = "root"
PG_PASSWORD = "your-password"
PG_HOST = "db"  # e.g., localhost, 127.0.0.1
PG_PORT = "5432" # default port for postgresql
PG_DATABASE = "postgres"

PG_URL = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"


engine = create_engine(PG_URL)
SQLModel.metadata.create_all(engine)


InvalidRequestError: Table 'event' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [32]:
from datetime import datetime

# Create an instance of the Event model with a datetime value
event = Event(
    name="Product Launch",
    description="Launch of new product",
    event_time=datetime.now()  # Year, Month, Day, Hour, Minute
)

# Add the event to the database
with Session(engine) as session:
    session.add(event)
    session.commit()
    session.refresh(event)

print(f"Event created: {event}")

Event created: description='Launch of new product' event_time=datetime.datetime(2025, 2, 3, 16, 0, 42, 734206) id=2 name='Product Launch'


In [1]:
from sqlmodel import SQLModel, Field
from datetime import datetime
from sqlalchemy.sql import func
from sqlalchemy.exc import OperationalError
from sqlmodel import create_engine, Session

# Define the Event model
class Event(SQLModel, table=True):
    id: int = Field(default=None, primary_key=True)
    name: str
    description: str
    event_time: datetime

    # Automatically set the creation timestamp
    created_at: datetime = Field(default_factory=datetime.utcnow, nullable=False)

    # Automatically update the timestamp on each update
    updated_at: datetime = Field(
        sa_column_kwargs={"server_default": func.now(), "onupdate": func.now()},
        default_factory=datetime.utcnow,
        nullable=False
    )

# Database configuration
PG_USER = "root"
PG_PASSWORD = "your-password"
PG_HOST = "db"  # e.g., localhost, 127.0.0.1
PG_PORT = "5432"  # Default port for PostgreSQL
PG_DATABASE = "postgres"

PG_URL = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"

# Create the engine
engine = create_engine(PG_URL)

# Create tables
try:
    SQLModel.metadata.create_all(engine)
    print("Tables created successfully.")
except OperationalError as e:
    print(f"Database connection error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Example: Insert a row into the Event table
def create_event():
    event = Event(name="Test Event", description="This is a test", event_time=datetime.utcnow())
    with Session(engine) as session:
        session.add(event)
        try:
            session.commit()
            session.refresh(event)
            print(f"Event created: {event}")
        except Exception as e:
            print(f"Error inserting event: {e}")

if __name__ == "__main__":
    create_event()

Tables created successfully.
Event created: id=1 name='Test Event' event_time=datetime.datetime(2025, 2, 3, 20, 28, 25, 689162) updated_at=datetime.datetime(2025, 2, 3, 20, 28, 25, 690524) description='This is a test' created_at=datetime.datetime(2025, 2, 3, 20, 28, 25, 690522)


In [3]:
event = Event(name="Test Event 2", description="This is a test 2", event_time=datetime.utcnow())
with Session(engine) as session:
    session.add(event)
    try:
        session.commit()
        session.refresh(event)
        print(f"Event created: {event}")
    except Exception as e:
        print(f"Error inserting event: {e}")

Event created: id=2 name='Test Event 2' event_time=datetime.datetime(2025, 2, 3, 22, 22, 57, 710486) updated_at=datetime.datetime(2025, 2, 3, 22, 22, 57, 711891) description='This is a test 2' created_at=datetime.datetime(2025, 2, 3, 22, 22, 57, 711889)


In [8]:
def update_event(event_id: int, updated_event: Event):
    with Session(engine) as session:
        event = session.get(Event, event_id)
        if event:
            event.name = updated_event.name
            event.description = updated_event.description
            event.event_time = updated_event.event_time
            session.add(event)
            session.commit()
            session.refresh(event)
            print(f"Event updated: {event}")
        else:
            print(f"Product with ID {event_id} not found.")

In [10]:
event = Event(name="Ben", description="Bensh_2", event_time=datetime.utcnow())
update_event(event_id=1,updated_event=event)

Event updated: id=1 name='Ben' event_time=datetime.datetime(2025, 2, 3, 22, 31, 42, 402387) updated_at=datetime.datetime(2025, 2, 3, 22, 31, 42, 407742) description='Bensh_2' created_at=datetime.datetime(2025, 2, 3, 20, 28, 25, 690522)
